In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_5251/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import logging
import pytorch_lightning as pl
import warnings

warnings.filterwarnings('ignore')
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

## Data load

In [3]:
! mkdir ../../data
! curl -OL https://storage.yandexcloud.net/di-datasets/age-prediction-nti-sbebank-2019.zip
! unzip -j -o age-prediction-nti-sbebank-2019.zip 'data/*.csv' -d ../../data
! mv age-prediction-nti-sbebank-2019.zip ../../data/

mkdir: cannot create directory ‘../../data’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  239M  100  239M    0     0  43.6M      0  0:00:05  0:00:05 --:--:-- 50.2M
Archive:  age-prediction-nti-sbebank-2019.zip
  inflating: ../../data/test.csv     
  inflating: ../../data/small_group_description.csv  
  inflating: ../../data/train_target.csv  
  inflating: ../../data/transactions_train.csv  
  inflating: ../../data/transactions_test.csv  


## Pyspark Data Preproccessing

In [4]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T


data_path = '../../data/'

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "16g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/19 08:27:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/19 08:27:12 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


[('spark.driver.memoryOverhead', '4g'),
 ('spark.app.id', 'local-1650356833093'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir',
  'file:/mnt/mikheev/pytorch-lifestream/demo/spark-warehouse'),
 ('spark.driver.memory', '16g'),
 ('spark.driver.port', '41019'),
 ('spark.executor.memory', '16g'),
 ('spark.app.startTime', '1650356832091'),
 ('spark.local.dir', '../../spark_cache'),
 ('spark.driver.host', 'tdl3.internal.cloudapp.net'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.sql.shuffle.partitions', '200'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'PysparkDataPreprocessor'),
 ('spark.cores.max', '24'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.maxResultSize', '4g'),
 ('spark.executor.memoryOverhead', '4g')]

In [6]:
source_data = spark.read.options(header=True, inferSchema=True).csv(os.path.join(data_path, 'transactions_train.csv'))
source_data.show(2)

+---------+----------+-----------+----------+
|client_id|trans_date|small_group|amount_rur|
+---------+----------+-----------+----------+
|    33172|         6|          4|    71.463|
|    33172|         6|         35|    45.017|
+---------+----------+-----------+----------+
only showing top 2 rows



In [7]:
from dltranz.data_preprocessing import PysparkDataPreprocessor

preprocessor = PysparkDataPreprocessor(
    col_id='client_id',
    cols_event_time='trans_date',
    time_transformation='float',
    cols_category=["trans_date", "small_group"],
    cols_log_norm=["amount_rur"],
    cols_identity=[],
    print_dataset_info=False,
)

In [8]:
%%time

dataset_pysparkdf = preprocessor.fit_transform(source_data).persist()
dataset_pysparkdf.count()

22/04/19 08:27:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/19 08:27:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/19 08:27:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/19 08:27:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/19 08:27:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/19 08:27:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/04/19 0

CPU times: user 475 ms, sys: 226 ms, total: 701 ms
Wall time: 29.2 s


30000

In [9]:
dataset_pysparkdf.show()

+---------+--------------------+--------------------+--------------------+--------------------+
|client_id|          trans_date|         small_group|          amount_rur|          event_time|
+---------+--------------------+--------------------+--------------------+--------------------+
|      463|[726, 724, 725, 7...|[7, 1, 7, 3, 3, 2...|[0.32135927726085...|[1.0, 2.0, 5.0, 7...|
|      471|[730, 726, 726, 7...|[68, 5, 4, 7, 1, ...|[0.26422890225828...|[0.0, 1.0, 1.0, 1...|
|      496|[723, 723, 722, 7...|[3, 1, 1, 1, 1, 4...|[0.27119266729746...|[3.0, 3.0, 4.0, 5...|
|      833|[726, 726, 726, 7...|[17, 15, 1, 49, 3...|[0.28922101008224...|[1.0, 1.0, 1.0, 2...|
|     1238|[730, 723, 716, 7...|[3, 11, 3, 3, 1, ...|[0.13326234667635...|[0.0, 3.0, 9.0, 1...|
|     1342|[730, 722, 702, 7...|[14, 3, 3, 21, 37...|[0.11547227625749...|[0.0, 4.0, 11.0, ...|
|     1591|[724, 724, 723, 7...|[37, 20, 7, 6, 1,...|[0.35099852868535...|[2.0, 2.0, 3.0, 3...|
|     1645|[724, 723, 723, 7...|[2, 16, 

In [10]:
dataset_pysparkdf.dtypes

[('client_id', 'int'),
 ('trans_date', 'array<int>'),
 ('small_group', 'array<int>'),
 ('amount_rur', 'array<double>'),
 ('event_time', 'array<float>')]

In [13]:
N = dataset_pysparkdf.count()
print('Size of whole dataset:', N)

test_df = dataset_pysparkdf.limit(int(0.2 * N))
train_df = dataset_pysparkdf.subtract(test_df)

N_train = train_df.count()
valid_df = train_df.limit(int(0.1 * N_train))
train_df = train_df.subtract(valid_df)

print('Size of test dataset:', test_df.count())
print('Size of train dataset', train_df.count())
print('Size of valid dataset', valid_df.count())

test_df.write.parquet('test.parquet')
train_df.write.parquet('train.parquet')
valid_df.write.parquet('valid.parquet')

Size of whole dataset: 30000


Size of test dataset: 6000


Size of train dataset 21600


Size of valid dataset 2400


## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (pl.LightningModule)
    * data_module (pl.LightningDataModule)
    * pl.trainer (pl.trainer)
    
For futher details check https://www.pytorchlightning.ai/

### model 

In [14]:
from dltranz.seq_encoder import SequenceEncoder
from dltranz.models import Head
from dltranz.lightning_modules.emb_module import EmbModule

seq_encoder = SequenceEncoder(
    category_features=preprocessor.get_category_sizes(),
    numeric_features=["amount_rur"],
    trx_embedding_noize=0.003
)

head = Head(input_size=seq_encoder.embedding_size, use_norm_encoder=True)

model = EmbModule(seq_encoder=seq_encoder, head=head)

### Data module

In [15]:
from dltranz.data_load.data_module.emb_data_module import EmbeddingTrainDataModule

dm_parquet = EmbeddingTrainDataModule(
    pl_module=model,
    parquet_train_path='train.parquet',
    parquet_valid_path='valid.parquet',
    min_seq_len=25,
    seq_split_strategy='SampleSlices',
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
    split_count=5,
    split_cnt_min=25,
    split_cnt_max=200,
    train_num_workers=16,
    train_batch_size=256,
    valid_num_workers=16,
    valid_batch_size=256
)

### Trainer

In [16]:
import torch
import pytorch_lightning as pl

import logging
# logging.getLogger("lightning").addHandler(logging.NullHandler())
# logging.getLogger("lightning").propagate = False

trainer = pl.Trainer(
#     progress_bar_refresh_rate=0,
    max_epochs=150,
    gpus=1 if torch.cuda.is_available() else 0
)

### Training 

In [1]:
%%time

trainer.fit(model, dm_parquet)

## Inference 

In [47]:
train = train_df.toPandas().to_dict('records')
for d in train:
    for k, v in d.items():
        if isinstance(v, list):
            d[k] = np.array(v)

test = test_df.toPandas().to_dict('records')
for d in test:
    for k, v in d.items():
        if isinstance(v, list):
            d[k] = np.array(v)

In [49]:
# embedding inference

from dltranz.inference import get_embeddings

train_embeds = get_embeddings(
    data=train,
    model=model, 
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
)

test_embeds = get_embeddings(
    data=test,
    model=model, 
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
)

train_embeds.shape, test_embeds.shape

((21600, 512), (6000, 512))

In [50]:
# join target and embeddings

df_target = pd.read_csv(os.path.join(data_path, 'train_target.csv'))
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)
train_df.head(2)

(21600, 514) (6000, 514)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_504,embed_505,embed_506,embed_507,embed_508,embed_509,embed_510,embed_511,client_id,target
0,0.800337,0.058391,0.025423,-0.186267,0.035758,-0.237643,0.052041,-0.041745,0.517471,-0.240486,...,0.278272,-0.920710,-0.947493,-0.137431,-0.052345,0.014428,-0.975207,-0.129017,48529,3
1,0.977548,-0.066553,0.030726,-0.156769,-0.252864,-0.275211,0.059669,-0.005294,0.546271,0.749741,...,0.238649,-0.956998,-0.916645,-0.119243,-0.045673,0.026230,-0.968116,0.161983,48852,3


Obtained embeddings can be used as features for model training

For example:

In [51]:
from sklearn.linear_model import LogisticRegression

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = LogisticRegression()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.6221666666666666